In [ ]:
import torch

# Free up GPU memory
torch.cuda.empty_cache()

# Now you can proceed with initializing your model and training


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers[torch]
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

from transformers import Trainer, TrainingArguments
from datasets import Dataset
from datasets import load_dataset




In [ ]:

# Load the pre-trained models and tokenizers
roberta_model = AutoModelForSequenceClassification.from_pretrained("roberta-base")
roberta_tokenizer = AutoTokenizer.from_pretrained("roberta-base")
# print memory footprint
print("Memory footprint of roberta_model: ", roberta_model.num_parameters() * 4 / 1024 / 1024, "MB")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Memory footprint of roberta_model:  475.49121856689453 MB


In [ ]:
# Clear the cache
from datasets import load_dataset



# Load the dataset from CSV files
dataset= load_dataset("csv", data_files={"train": "train.csv", "test": "test.csv"})

# Tokenize the text data
def tokenize_function(example):
    return roberta_tokenizer(example["review"], padding=True, truncation=True)

# Tokenize train and test datasets
train_dataset = dataset["train"].map(tokenize_function, batched=True)
test_dataset = dataset["test"].map(tokenize_function, batched=True)

# Optionally, convert labels to numerical format if necessary
# For example, if sentiment labels are strings ("positive", "negative"), convert them to integers (0, 1)
train_dataset = train_dataset.map(lambda examples: {"label": 1 if examples["sentiment"] == "positive" else 0})
test_dataset = test_dataset.map(lambda examples: {"label": 1 if examples["sentiment"] == "positive" else 0})





Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from transformers import AdamW
from tqdm import tqdm

# Define the model configuration
MODEL_NAME = 'roberta-base'
NUM_LABELS = 2  # Assuming binary classification (positive and negative)

# Load the pre-trained RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
model = RobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_LABELS)

# Prepare the tokenized training data
input_ids = torch.tensor(train_dataset['input_ids'])
attention_masks = torch.tensor(train_dataset['attention_mask'])

# Convert sentiment labels to numerical format
label_map = {"positive": 1, "negative": 0}  # Define a mapping from string labels to numerical labels
labels = torch.tensor([label_map[sentiment] for sentiment in train_dataset['sentiment']])

# Define the batch size and create DataLoader
BATCH_SIZE = 8
train_dataset = TensorDataset(input_ids, attention_masks, labels)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

# Specify GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Fine-tuning the model
EPOCHS = 3

for epoch in range(EPOCHS):
    model.train()

    # Initialize progress bar
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{EPOCHS}', leave=False, disable=False)

    for batch in progress_bar:
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}

        optimizer.zero_grad()

        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()

        optimizer.step()

        progress_bar.set_postfix({'training_loss': f'{loss.item():.3f}'})

    # Evaluate the model if needed
    # ...

# Save the fine-tuned model
model.save_pretrained('/content/drive/My Drive/fine_tuned_roberta_model')
tokenizer.save_pretrained('/content/drive/My Drive/fine_tuned_roberta_model')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


('/content/drive/My Drive/fine_tuned_roberta_model/tokenizer_config.json',
 '/content/drive/My Drive/fine_tuned_roberta_model/special_tokens_map.json',
 '/content/drive/My Drive/fine_tuned_roberta_model/vocab.json',
 '/content/drive/My Drive/fine_tuned_roberta_model/merges.txt',
 '/content/drive/My Drive/fine_tuned_roberta_model/added_tokens.json')

In [ ]:
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

# Load the test data
test_df = pd.read_csv("test.csv")

# Load the pre-trained DistilBERT tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('/content/drive/My Drive/fine_tuned_roberta_model')
model = RobertaForSequenceClassification.from_pretrained('/content/drive/My Drive/fine_tuned_roberta_model')

# Tokenize the test data
test_encodings = tokenizer(list(test_df['review']), truncation=True, padding=True, return_tensors='pt')

# Convert the sentiment labels to numerical format
label_map = {"positive": 1, "negative": 0}
test_labels = [label_map[label] for label in test_df['sentiment']]

# Create a TensorDataset
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))

# DataLoader for test dataset
batch_size = 8
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Evaluate the model
model.eval()
predictions = []
true_labels = []

for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predictions.extend(torch.argmax(logits, dim=1).cpu().tolist())
    true_labels.extend(inputs['labels'].cpu().tolist())

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.9413


In [ ]:
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

# Load the test data
test_df = pd.read_csv("test.csv", error_bad_lines=False)

# Load the pre-trained DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# Tokenize the test data
test_encodings = tokenizer(list(test_df['review']), truncation=True, padding=True, return_tensors='pt')

# Convert the sentiment labels to numerical format
label_map = {"positive": 1, "negative": 0}
test_labels = [label_map[label] for label in test_df['sentiment']]

# Create a TensorDataset
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))

# DataLoader for test dataset
batch_size = 8
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Evaluate the model
model.eval()
predictions = []
true_labels = []

for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predictions.extend(torch.argmax(logits, dim=1).cpu().tolist())
    true_labels.extend(inputs['labels'].cpu().tolist())

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Test Accuracy: {accuracy}")


<ipython-input-5-e34ed8bfc253>:8: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  test_df = pd.read_csv("test.csv", error_bad_lines=False)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Test Accuracy: 0.8891
